## Importing python packages for the analysis of data.

### In this analysis, I will be using following packages:
1. Python v3.0
2. Numpy v1.12.1
3. Pandas v0.20.1
4. Plotly v2.0.8
5. Cufflinks v0.8.2


In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from helper.calc import *
#import cufflinks as cf

#plotly.offline.init_notebook_mode() # run at the start of every notebook
plotly.tools.set_credentials_file(username='pankajs.phy', api_key='e0zYzuRpN5PC3GZV4zYC')
#plotly.tools.set_config_file(world_readable=True)

### First I am importing the Excel data into the workspace in panda. The imported data will be stored in a pandas dataframe.
### In the next step, all the column headings will be renamed. Basically, the whitespaces in the headings are replaced with an underscore.

In [2]:
xl = pd.ExcelFile("./data/Consumer_1_profile_2010-2011.xlsx",index=False)
df = xl.parse("Sheet1",index=False)
df=df.rename(columns={"Customer no." : "Customer_No", 'Consumption Category' : 'Consumption_Category','Period 1':'00:00','Period 2':'00:30','Period 3':'01:00','Period 4':'01:30','Period 5':'02:00','Period 6':'02:30','Period 7':'03:00','Period 8':'03:30','Period 9':'04:00','Period 10':'04:30','Period 11':'05:00','Period 12':'05:30','Period 13':'06:00','Period 14':'06:30','Period 15':'07:00','Period 16':'07:30','Period 17':'08:00','Period 18':'08:30','Period 19':'09:00','Period 20':'09:30','Period 21':'10:00','Period 22':'10:30','Period 23':'11:00','Period 24':'11:30','Period 25':'12:00','Period 26':'12:30','Period 27':'13:00','Period 28':'13:30','Period 29':'14:00','Period 30':'14:30','Period 31':'15:00','Period 32':'15:30','Period 33':'16:00','Period 34':'16:30','Period 35':'17:00','Period 36':'17:30','Period 37':'18:00','Period 38':'18:30','Period 39':'19:00','Period 40':'19:30','Period 41':'20:00','Period 42':'20:30','Period 43':'21:00','Period 44':'21:30','Period 45':'22:00','Period 46':'22:30','Period 47':'23:00','Period 48':'23:30'})
df.head();

### In the following code, the dataframe is splitted into three dataframes according to consumption category i.e., df_GG, df_CL and df_GC.


In [3]:
df_GG=df[df.Consumption_Category=='GG']
df_CL=df[df.Consumption_Category=='CL']
df_GC=df[df.Consumption_Category=='GC']

In [4]:
df_GG.to_csv('./data/Consumer_1_2010-2011_GG.csv', encoding='utf-8',index=False)
df_CL.to_csv('./data/Consumer_1_2010-2011_CL.csv', encoding='utf-8',index=False)
df_GC.to_csv('./data/Consumer_1_2010-2011_GC.csv', encoding='utf-8',index=False)

In [5]:
df_GG = pd.read_csv('./data/Consumer_1_2010-2011_GG.csv',
                    parse_dates={'Date': [2,3,4]}, 
                    date_parser=lambda x: pd.datetime.strptime(x, '%d %m %Y'))


df_CL = pd.read_csv('./data/Consumer_1_2010-2011_CL.csv',
                    parse_dates={'Date': [2,3,4]}, 
                    date_parser=lambda x: pd.datetime.strptime(x, '%d %m %Y'))


df_GC = pd.read_csv('./data/Consumer_1_2010-2011_GC.csv',
                    parse_dates={'Date': [2,3,4]}, 
                    date_parser=lambda x: pd.datetime.strptime(x, '%d %m %Y'))

In [11]:
from helper.interpolate import interpol

#First Off-Peak Losses

order = 1

yi=[16.76,10.89,6.36,4.41,3.10,2.17,1.48,0.99,0.67,0.30]
xi=[0.03,0.06,0.09,0.11,0.14,0.17,0.20,0.23,0.26,0.29]
x=np.linspace(0,0.309,2000)

df_loss_off_peak = pd.DataFrame({})
df_loss_off_peak["Off_Peak_Consumption"]=x
df_loss_off_peak["Loss"]=interpol(xi,yi,x,order)
df_loss_off_peak["Loss"][0]=0.0

#Now Peak Consumption Losses with extrapolation

yi = [1.79,2.92,3.55,4.24]
xi = [0.46,0.49,0.51,0.54]
x  = np.linspace(0.46,2.5,2000)

df_loss_peak_geom = pd.DataFrame({})
df_loss_peak_geom["Peak_Consumption"]=x
df_loss_peak_geom["Loss"]=interpol(xi,yi,x,order)

#Now Peak Consumption Losses without extrapolation
yi = [1.79,2.92,3.55,4.24]
xi = [0.46,0.49,0.51,0.54]
x  = np.linspace(0.45,0.54,1000)
y = interpol(xi,yi,x,order)

x = np.append(x, np.array(np.linspace(0.540001,2.5,1000)))
y = np.append(y, [4.24 for i in range(0,1000)])

df_loss_peak = pd.DataFrame({})
df_loss_peak["Peak_Consumption"]=x
df_loss_peak["Loss"]=y


### Here we create a new dataframe which contains the volume of discharge needed in each period for consumer1.

In [6]:
start = datetime.datetime(2010, 7, 1) #Start of the year
end   = datetime.datetime(2011, 6, 30)#End of the year
Dates = pd.date_range(start, end)#Series of the dates in the year 2010-2011

df_vol_dschg = pd.DataFrame() #New dataframe for volme discharge
df_vol_dschg['Date'] = Dates # Creating first column with dates

df_consump = pd.DataFrame() #New dataframe for volme discharge
df_consump['Date'] = Dates # Creating first column with dates

for i in range(0,48): #Function to create new columns corresponding to volume discharge    
    if(i%2 == 0):
        if (i/2 <= 9):
            key = '0'+str(int(i/2))+':00'
        elif (i/2 >=10):
            key = str(int(i/2))+':00'
    elif(i%2 == 1):
        if (i/2 < 10):
            key = '0'+str(int(i/2))+':30'
        elif (i/2 >=10):
            key = str(int(i/2))+':30'
            
    df_vol_dschg[key] = dch_vol(df_GC,df_CL,df_GG,key); #Calling the function dch_vol to provide values to Period_1 to Period_48            
    df_consump[key] = consump(df_GC,df_CL,key); #Calling the function consump to provide values to Period_1 to Period_48                

# Converting Date column as datetime data type    
df_vol_dschg['Date'] = pd.to_datetime(Dates) 
df_vol_dschg.index = df_vol_dschg['Date']
#df_vol_dschg["Total"] = df_vol_dschg.iloc[:,1:49].sum(axis=1)
df_vol_dschg=df_vol_dschg.drop('Date',axis=1)

# Writing the new dataframe into csv file
df_vol_dschg.to_csv('./data/Consumer_1_2010-2011_discharge.csv', encoding='utf-8',index=False);

# Converting Date column as datetime data type    
df_consump['Date'] = pd.to_datetime(Dates) 
df_consump.index = df_consump['Date']
#df_consump["Total"] = df_consump.iloc[:,1:49].sum(axis=1)
df_consump=df_consump.drop('Date',axis=1)
# Writing the new dataframe into csv file
df_consump.to_csv('./data/Consumer_1_2010-2011_consumption.csv', encoding='utf-8',index=False);
df_consump.head();

In [7]:
Monthly_std_discharge = df_vol_dschg.resample('M').std()
Monthly_mean_discharge = df_vol_dschg.resample('M').mean()
Monthly_min_discharge  = df_vol_dschg.resample('M').min()
Monthly_max_discharge  = df_vol_dschg.resample('M').max()

Monthly_mean_discharge.describe();
Monthly_std_discharge.iloc[:,1:10];

In [8]:
Trans_Monthly_mean = Monthly_mean_discharge.transpose();
Trans_Monthly_min  = Monthly_min_discharge.transpose();
Trans_Monthly_max  = Monthly_max_discharge.transpose();
Trans_Monthly_std  = Monthly_std_discharge.transpose();

In [9]:
from helper.plot_plotly import plot_shaded
ytitle = 'Mean half-hourly losses (cents)'
plot_shaded(df_vol_dschg,'December',ytitle)

In [10]:
from helper.plot_plotly import plot_multiplot
Title='Mean variation of discharge volume across a financial year 2010-2011'
#plot_multiplot(df_vol_dschg,Title)

In [11]:
from helper.interpolate import interpol

#First Off-Peak Losses

order = 1

yi=[16.76,10.89,6.36,4.41,3.10,2.17,1.48,0.99,0.67,0.30]
xi=[0.03,0.06,0.09,0.11,0.14,0.17,0.20,0.23,0.26,0.29]
x=np.linspace(0,0.309,2000)

df_loss_off_peak = pd.DataFrame({})
df_loss_off_peak["Off_Peak_Consumption"]=x
df_loss_off_peak["Loss"]=interpol(xi,yi,x,order)
df_loss_off_peak["Loss"][0]=0.0

#Now Peak Consumption Losses with extrapolation

yi = [1.79,2.92,3.55,4.24]
xi = [0.46,0.49,0.51,0.54]
x  = np.linspace(0.46,2.5,2000)

df_loss_peak_geom = pd.DataFrame({})
df_loss_peak_geom["Peak_Consumption"]=x
df_loss_peak_geom["Loss"]=interpol(xi,yi,x,order)

#Now Peak Consumption Losses without extrapolation
yi = [1.79,2.92,3.55,4.24]
xi = [0.46,0.49,0.51,0.54]
x  = np.linspace(0.45,0.54,1000)
y = interpol(xi,yi,x,order)

x = np.append(x, np.array(np.linspace(0.540001,2.5,1000)))
y = np.append(y, [4.24 for i in range(0,1000)])

df_loss_peak = pd.DataFrame({})
df_loss_peak["Peak_Consumption"]=x
df_loss_peak["Loss"]=y


In [12]:
Monthly_std_consump  = df_consump.resample('M').std()
Monthly_mean_consump = df_consump.resample('M').mean()
Monthly_min_consump  = df_consump.resample('M').min()
Monthly_max_consump  = df_consump.resample('M').max()

Monthly_mean_consump.describe();
Monthly_std_consump.transpose().iloc[:,1:10];

In [13]:
from helper.interpolate import cost_consump
Monthly_mean_cost = pd.DataFrame.copy(Monthly_mean_consump,deep=True)
Monthly_mean_cost_geom = pd.DataFrame.copy(Monthly_mean_consump,deep=True)

Monthly_mean_cost = Monthly_mean_consump.transpose().applymap(lambda x: cost_consump(x,df_loss_peak,df_loss_off_peak))
Monthly_mean_cost_geom = Monthly_mean_consump.applymap(lambda x: cost_consump(x,df_loss_peak_geom,df_loss_off_peak))


ImportError: No module named 'interpolate'

In [ ]:
Title='Mean Consumption losses across a financial year 2010-2011'
figname = './plots/Mean_Cost.png'
plot_multiplot(Monthly_mean_cost_geom,Title,figname)

In [ ]:
Title='Mean Consumption losses across a financial year 2010-2011'
figname = './plots/Mean_Cost.png'
plot_multiplot(Monthly_mean_cost_geom,Title,figname)